# Gradio (ML Models interfaces)

## Installation

In [1]:
!pip install gradio
!pip install transformers

## Image Classification

In [2]:
import pandas as pd
import numpy as np
import gradio as gr
import torch
import torchvision.models as models
from torchvision import transforms
import requests
from PIL import Image

alexnet = models.alexnet(pretrained=True).eval()
raw_labels = requests.get("https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt").text
labels = [i.split(',')[0] for i in list(eval(raw_labels).values())]

def inference(data):
    image = transforms.ToTensor()(Image.fromarray(data.astype('uint8'), 'RGB')).unsqueeze(0)
    prediction = torch.nn.functional.softmax(alexnet(image)[0], dim=0)
    dictionary = dict(zip(labels, map(float, prediction)))
    return dictionary

gr.Interface(fn=inference, 
             inputs=gr.inputs.Image(), 
             outputs=gr.outputs.Label(num_top_classes=5)).launch(share=True) #, debug=True Use in Colab

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://57365.gradio.app
Interface loading below...


(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x7f5324bfd898>,
 'http://127.0.0.1:7860/',
 'https://57365.gradio.app')

## Text Generation

In [3]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Loading and setting up GPT2 Open AI Transformer Model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", 
                                          pad_token_id=tokenizer.eos_token_id)

def generate_text(inp):
    # Encoding the starting point of the sentence we want to predict
    input_data = tokenizer.encode(inp, 
                                  return_tensors='tf')
    # Generating Output String
    output = model.generate(
        input_data, 
        do_sample=True, 
        max_length=50, 
        top_k=30
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

gr.Interface(generate_text,
             "textbox", 
             gr.outputs.Textbox()).launch(share=True) #, debug=True Use in Colab

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://14892.gradio.app
Interface loading below...


(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x7f52e0702c18>,
 'http://127.0.0.1:7861/',
 'https://14892.gradio.app')

## Live General ML model prediction

In [4]:
# Preprocessing :
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix
from itertools import product

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [5]:
df = pd.read_csv("heart.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [6]:
df = df.drop(['cp','trestbps', 'fbs', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal'], axis=1)
df.head()

,age,sex,chol,restecg,target
0,63,1,233,0,1
1,37,1,250,1,1
2,41,0,204,0,1
3,56,1,236,1,1
4,57,0,354,1,1


In [7]:
df['age'].min(), df['age'].max()

(29, 77)

In [8]:
df['sex'].min(), df['sex'].max()

(0, 1)

In [9]:
df['chol'].min(), df['chol'].max()

(126, 564)

In [10]:
df['restecg'].min(), df['restecg'].max()

(0, 2)

In [11]:
X = df.drop(['target'], axis = 1).values
Y = df['target']

st = StandardScaler()
X = st.fit_transform(X)

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.30, random_state = 101)

In [12]:
trainedmodel = LogisticRegression().fit(X_Train,Y_Train)
predictions =trainedmodel.predict(X_Test)
print(confusion_matrix(Y_Test,predictions))
print(classification_report(Y_Test,predictions))

[[28 16]
 [13 34]]
              precision    recall  f1-score   support

           0       0.68      0.64      0.66        44
           1       0.68      0.72      0.70        47

    accuracy                           0.68        91
   macro avg       0.68      0.68      0.68        91
weighted avg       0.68      0.68      0.68        91



In [13]:
trainedforest = RandomForestClassifier(n_estimators=700).fit(X_Train,Y_Train)
predictionforest = trainedforest.predict(X_Test)
print(confusion_matrix(Y_Test,predictionforest))
print(classification_report(Y_Test,predictionforest))

[[23 21]
 [22 25]]
              precision    recall  f1-score   support

           0       0.51      0.52      0.52        44
           1       0.54      0.53      0.54        47

    accuracy                           0.53        91
   macro avg       0.53      0.53      0.53        91
weighted avg       0.53      0.53      0.53        91



In [14]:
trainedsvm = svm.LinearSVC().fit(X_Train, Y_Train)
predictionsvm = trainedsvm.predict(X_Test)
print(confusion_matrix(Y_Test,predictionsvm))
print(classification_report(Y_Test,predictionsvm))

[[28 16]
 [13 34]]
              precision    recall  f1-score   support

           0       0.68      0.64      0.66        44
           1       0.68      0.72      0.70        47

    accuracy                           0.68        91
   macro avg       0.68      0.68      0.68        91
weighted avg       0.68      0.68      0.68        91



In [15]:
trainedtree = tree.DecisionTreeClassifier().fit(X_Train, Y_Train)
predictionstree = trainedtree.predict(X_Test)
print(confusion_matrix(Y_Test,predictionstree))
print(classification_report(Y_Test,predictionstree))

[[26 18]
 [23 24]]
              precision    recall  f1-score   support

           0       0.53      0.59      0.56        44
           1       0.57      0.51      0.54        47

    accuracy                           0.55        91
   macro avg       0.55      0.55      0.55        91
weighted avg       0.55      0.55      0.55        91



In [16]:
trainedlda = LinearDiscriminantAnalysis().fit(X_Train, Y_Train)
predictionlda = trainedlda.predict(X_Test)
print(confusion_matrix(Y_Test,predictionlda))
print(classification_report(Y_Test,predictionlda))

[[28 16]
 [13 34]]
              precision    recall  f1-score   support

           0       0.68      0.64      0.66        44
           1       0.68      0.72      0.70        47

    accuracy                           0.68        91
   macro avg       0.68      0.68      0.68        91
weighted avg       0.68      0.68      0.68        91



In [17]:
trainednb = GaussianNB().fit(X_Train, Y_Train)
predictionnb = trainednb.predict(X_Test)
print(confusion_matrix(Y_Test,predictionnb))
print(classification_report(Y_Test,predictionnb))

[[31 13]
 [14 33]]
              precision    recall  f1-score   support

           0       0.69      0.70      0.70        44
           1       0.72      0.70      0.71        47

    accuracy                           0.70        91
   macro avg       0.70      0.70      0.70        91
weighted avg       0.70      0.70      0.70        91



In [18]:
def predict_survival(age, sex, ch, cardio):
    if sex=='female':
      s=0
    else:
      s=1
    df = pd.DataFrame.from_dict({'Age': [age], 'Sex': [s], 
                                 'Cholestoral (in mg/dl)': [ch], 
                                 'Resting electrocardiographic results': [cardio]})
    df = st.transform(df)
    pred = trainedmodel.predict_proba(df)[0]
    return {'No Heart Desease': pred[0], 'Has Heart Desease': pred[1]}

sex = gr.inputs.Radio(['female', 'male'], label="Sex")
age = gr.inputs.Slider(minimum=1, maximum=100, default=22, label="Age")
ch = gr.inputs.Slider(minimum=120, maximum=560, default=200, label="Cholestoral (in mg/dl)")
cardio = gr.inputs.Radio([0, 1, 2], label="Resting electrocardiographic results")

gr.Interface(predict_survival, [age, sex, ch, cardio], "label", live=True).launch(share=True) #, debug=True Use in Colab

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://14261.gradio.app
Interface loading below...


(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x7f52e06c7e80>,
 'http://127.0.0.1:7862/',
 'https://14261.gradio.app')